In [90]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import os
import re

In [1]:
def get_trm():
    response = requests.get('https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/latest/currencies/usd/cop.json')

    if response.status_code == 200:
        data = response.json()
        return (data['cop']) # Imprime el valor de la tasa de cambio
    else:
        print("Ha ocurrido un error al hacer la solicitud.")
        return 5000
        

In [4]:
##Leer la pagina de busqueda para los 16 forros mas buscados ordenados por pedidos y envio gratis si supera la marca
browser = uc.Chrome()

url = 'https://www.amazon.com/s?k=phone+cases&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&language=es_US&ds=v1%3ASqcA0tcrOG1e%2Bc8XK79Se0V71IaypuNtYyHOuzdemHY&crid=W5XUJ7TR2HEQ&qid=1681159723&rnid=10236241011&sprefix=phone+case%2Caps%2C221&ref=sr_nr_p_n_is_free_shipping_1'
url2 = 'https://www.amazon.com/-/es/s?k=phone+cases&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&page=2&language=es_US&crid=W5XUJ7TR2HEQ&qid=1681443982&rnid=10236241011&sprefix=phone+case%2Caps%2C221&ref=sr_pg_2'
browser.get(url)

browser.implicitly_wait(60)

html = browser.page_source

soup = bs(html,'lxml')

browser.get(url2)

browser.implicitly_wait(60)

html2 = browser.page_source

soup2 = bs(html,'lxml')

#Cerrar pestaña y navegador
#browser.close()
#browser.quit()

In [8]:
#Crear array con las tarjetas de cada uno de los elementos (la tarjeta 0 es informacion)
cards = soup.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards2 = soup2.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards = cards + cards2


In [9]:
def extract_discount(text):
    # Definir la expresión regular
    regex = r"Ahorra\s+(\d+)\s*%"

    # Buscar el número usando la expresión regular
    match = re.search(regex, text)

    # Devolver el número como un entero si se encontró una coincidencia
    if match:
        return int(match.group(1))

    # Devolver None si no se encontró una coincidencia
    return None


In [10]:
#Definir funciones para extraer info primero de la card y luego del link de la card

def get_name(card):
    name = card.find('span', class_='a-size-medium a-color-base a-text-normal').text
    return name
def get_price(card):
    price = soup.find('span',{'class':'a-price-whole'}).text + soup.find('span',{'class':'a-price-fraction'}).text
    return float(price)*get_trm()
def get_discount(card):
    discount_text = soup.find('span',{'class':'a-size-base s-highlighted-text-padding aok-inline-block s-coupon-highlight-color'}).text
    discount = extract_discount(discount_text)
    return discount
def get_link(card):
    link = card.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    href = 'https://www.amazon.com.co' + link.get('href')
    return href
def get_link_soup(link):
    browser.get(link)
    browser.implicitly_wait(random.randint(30, 60))
    html = browser.page_source
    soup = bs(html,'lxml')
    return soup
    

75622.48036666999

In [11]:
def get_images(link_soup):
    lista = link_soup.find('ul', class_='a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro regularAltImageViewLayout')
    imagenes = list(map(lambda x: x.get('src'), lista.find_all('img')))
    cantidad = len(imagenes)-2
    script = soup.find_all("script", {"type": "text/javascript"})
    print(script)

    # Extraemos el contenido de la etiqueta script como una cadena de texto
    data_str = list(filter(lambda x: 'hiRes' in x.string.strip() , script))
    print(data_str)
    
def get_table(link_soup):
    tabla = link_soup.find('table',class_ = 'a-normal a-spacing-micro')
    tabla_dict = {}
    filas = tabla.find_all('tr')
    for fila in filas:
        celdas = fila.find_all('td')
        clave = celdas[0].text.strip()
        valor = celdas[1].text.strip()
        tabla_dict[clave] = valor
    return tabla_dict
    
    
#Creo que mercadolibre deja subir la descripcion como .md, por eso lo hago asi
def get_description(link_soup):
    description_soup = link_soup.find('div',{'id':'feature-bullets'})
    list_items = description_soup.find('ul').find_all('li')
    content = '\n'.join([f'* {li.text.strip()}' for li in list_items])
    
    # Obtener el título del artículo
    title = description_soup.find('h1').text.strip()
    
    # Combinar el título y el contenido y agregar separadores
    result = f'# {title}\n\n{content}\n\n---\n'
    
    return result
    



## Codigo para obtener imagenes de la pagina de cada case

In [15]:
#solo para la primera case, obtener soup
test = get_link_soup(get_link(cards[1]))

In [ ]:
script = test.find('div', class_="a-section imageBlockRearch").find_next_sibling('script')

import json

colorImages =(script.string)[script.string.index('colorImages')+14:script.string.index('}]}')+3].replace('null', 'None').replace("'", '"')

print(colorImages)



In [108]:
#Suponiendo que funciona diccio = dict(colorImages)
diccio = { 'initial': [{"hiRes":"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/51aELp0VZbL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/51aELp0VZbL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX342_.jpg":[342,342],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX385_.jpg":[385,385],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX425_.jpg":[425,425],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX466_.jpg":[466,466],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX522_.jpg":[522,522],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX569_.jpg":[569,569],"https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SX679_.jpg":[679,679]},"variant":"MAIN","lowRes":None,"shoppableScene":None},{"hiRes":"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/410fqzljLJL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/410fqzljLJL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX342_.jpg":[428,342],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX385_.jpg":[481,385],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX425_.jpg":[531,425],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX466_.jpg":[583,466],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX522_.jpg":[653,522],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX569_.jpg":[711,569],"https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SX679_.jpg":[849,679]},"variant":"PT01","lowRes":None,"shoppableScene":None},{"hiRes":"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/41e9GCwYXFL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/41e9GCwYXFL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX342_.jpg":[428,342],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX385_.jpg":[481,385],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX425_.jpg":[531,425],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX466_.jpg":[583,466],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX522_.jpg":[653,522],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX569_.jpg":[711,569],"https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SX679_.jpg":[849,679]},"variant":"PT02","lowRes":None,"shoppableScene":None},{"hiRes":"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/51qgJtei4SL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/51qgJtei4SL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX342_.jpg":[342,342],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX385_.jpg":[385,385],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX425_.jpg":[425,425],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX466_.jpg":[466,466],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX522_.jpg":[522,522],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX569_.jpg":[569,569],"https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SX679_.jpg":[679,679]},"variant":"PT03","lowRes":None,"shoppableScene":None},{"hiRes":"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/51vc-sjVykL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/51vc-sjVykL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX342_.jpg":[342,342],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX385_.jpg":[385,385],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX425_.jpg":[425,425],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX466_.jpg":[466,466],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX522_.jpg":[522,522],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX569_.jpg":[569,569],"https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SX679_.jpg":[679,679]},"variant":"PT04","lowRes":None,"shoppableScene":None},{"hiRes":"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SL1500_.jpg","thumb":"https://m.media-amazon.com/images/I/41MDJSouyWL._AC_SR38,50_.jpg","large":"https://m.media-amazon.com/images/I/41MDJSouyWL._AC_.jpg","main":{"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX342_.jpg":[342,342],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX385_.jpg":[385,385],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX425_.jpg":[425,425],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX466_.jpg":[466,466],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX522_.jpg":[522,522],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX569_.jpg":[569,569],"https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SX679_.jpg":[679,679]},"variant":"PT05","lowRes":None,"shoppableScene":None}]}
imagenes = diccio['initial']
imagenes_2 = [x['hiRes'] for x in imagenes]
imagenes_2

['https://m.media-amazon.com/images/I/71MIxZaKqgL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61Zv9iseeHL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71c8S3hqaUL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71h7ScgrLUL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71afmlGrdtL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61mm8+l95yL._AC_SL1500_.jpg']

In [ ]:
cases_array = []
i = 0
cantidad = len(cards)
browser = uc.Chrome()
for card in cards:
    i = i + 1
    print(str(i/cantidad*100) + '%')
    case_info = {}
    
    sopa = get_link_soup(get_link(cards[0]))
    case_info['name'] = get_name(card)
    case_info['price'] = get_price(card)
    case_info['discount'] = get_discount(card)
    case_info['link'] = get_link(card)    
    
    link_soup = get_link_soup(case_info['link'])
    
    #case_info['images'] = get_images(link_soup)
    case_info['table'] = get_table(link_soup)
    case_info['description'] = get_description(link_soup)
    
    cases_array.append(case_info)
    
    
#Cerrar pestaña y navegador
browser.close()
browser.quit()

cases_array
    

In [ ]:
list(map(lambda x: x['name'], cases_array))